# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
data.show(2)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|   Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|Wilson PLC|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
only showing top 2 rows



# Lidando a Coluna com timestamp

In [7]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [8]:
#data.select(year(data['Onboard_date'])).show()
data2=data.withColumn("Year",year(data['Onboard_date']))
data3=data2.withColumn("Month",month(data2['Onboard_date']))
data4=data3.withColumn("dayofmonth",month(data3['Onboard_date']))

In [9]:
data4.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----+-----+----------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|Year|Month|dayofmonth|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----+-----+----------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|2013|    8|         8|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|2013|    8|         8|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|2016|    6|         6|
|      Phillip W

In [10]:
data4.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'Year',
 'Month',
 'dayofmonth']

In [11]:
data4.schema

StructType(List(StructField(Names,StringType,true),StructField(Age,DoubleType,true),StructField(Total_Purchase,DoubleType,true),StructField(Account_Manager,IntegerType,true),StructField(Years,DoubleType,true),StructField(Num_Sites,DoubleType,true),StructField(Onboard_date,TimestampType,true),StructField(Location,StringType,true),StructField(Company,StringType,true),StructField(Churn,IntegerType,true),StructField(Year,IntegerType,true),StructField(Month,IntegerType,true),StructField(dayofmonth,IntegerType,true)))

In [12]:
my_cols = data4.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
# 'Onboard_date',
 'Location',
 'Company',
 'Year',
 'Month',
 'dayofmonth'])

In [13]:
from pyspark.sql.functions import isnan, when, count, col

#data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()
import pyspark.sql.functions as F
def count_missings(spark_df,sort=True):
    """
    Counts number of nulls and nans in each column
    """
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)

    return df

count_missings(data, False)

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,0,0,0,0,0,0


In [14]:
my_final_data = my_cols.na.drop()

In [15]:
## Working With Categorical Columns

In [16]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [17]:
location_indexer = StringIndexer(inputCol='Location',outputCol='LocationIndex')
#model = stringIndexer.fit(data4)
#indexed = model.transform(data4)
location_encoder = OneHotEncoder(inputCol='LocationIndex',outputCol='LocationVec')


In [18]:
company_indexer = StringIndexer(inputCol='Company',outputCol='CompanyIndex')
company_encoder = OneHotEncoder(inputCol='CompanyIndex',outputCol='CompanyVec')

In [19]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
# 'Onboard_date',
# 'LocationVec',
# 'CompanyVec',
 'Year',
 'Month',
 'dayofmonth'],outputCol='features')

In [20]:
#output = assembler.transform(data4)

# Using A Logistic Regression!

In [21]:
from pyspark.ml.classification import LogisticRegression

In [22]:
from pyspark.ml import Pipeline

In [23]:
log_reg_proj = LogisticRegression(featuresCol='features',labelCol='Churn')

In [24]:
pipeline_lr = Pipeline(stages=[location_indexer,company_indexer,
                           location_encoder, company_encoder,
                           assembler,log_reg_proj])

In [25]:
train_data, test_data =data4.randomSplit([0.7,.3])

In [26]:
fit_model = pipeline_lr.fit(train_data)

In [27]:
results = fit_model.transform(test_data)

In [28]:
results.select('prediction').show()

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 20 rows



In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [31]:

results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [32]:
my_eval.evaluate(results)

0.8503892060197198

In [33]:
# Melhorar o projeto: trabalhar com 'LocationVec' e 'CompanyVec',


# Testando uma Random Forest!

In [34]:
from pyspark.ml.classification import RandomForestClassifier

In [35]:
rf_proj = RandomForestClassifier(numTrees=300,featuresCol='features',labelCol='Churn')
#rf_proj_2 = RandomForestClassifier(numTrees=10,labelCol='Churn')

In [38]:
pipeline = Pipeline(stages=[location_indexer,company_indexer,
                           location_encoder, company_encoder,
                           assembler,rf_proj])

In [39]:
fit_model_rf = pipeline.fit(train_data)
#fit_model_rf = rf_proj_2.fit(train_data)

In [40]:
results = fit_model.transform(test_data)

In [41]:
results.select('prediction','Churn').show()

+----------+-----+
|prediction|Churn|
+----------+-----+
|       0.0|    0|
|       0.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       1.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 20 rows



In [42]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [43]:

results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [44]:
my_eval.evaluate(results)

0.8503892060197198

# Testing on new data!
## This test is useful to predict which clients will churn.

In [45]:
#final_lr_model = pipeline_lr.fit(final_data)

In [46]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [47]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [48]:
new_customers.head(16)

[Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date=datetime.datetime(2011, 8, 29, 18, 37, 54), Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd'),
 Row(Names='Michele Wright', Age=23.0, Total_Purchase=7526.94, Account_Manager=1, Years=9.28, Num_Sites=15.0, Onboard_date=datetime.datetime(2013, 7, 22, 18, 19, 54), Location='21083 Nicole Junction Suite 332, Youngport, ME 23686-4381', Company='Cannon-Benson'),
 Row(Names='Jeremy Chang', Age=65.0, Total_Purchase=100.0, Account_Manager=1, Years=1.0, Num_Sites=15.0, Onboard_date=datetime.datetime(2006, 12, 11, 7, 48, 13), Location='085 Austin Views Lake Julialand, WY 63726-4298', Company='Barron-Robertson'),
 Row(Names='Megan Ferguson', Age=32.0, Total_Purchase=6487.5, Account_Manager=0, Years=9.4, Num_Sites=14.0, Onboard_date=datetime.datetime(2016, 10, 28, 5, 32, 13), Location='922 Wright Branch North Cynthialand, NC 64721', Company='Sexton-

In [49]:
data2n=new_customers.withColumn("Year",year(new_customers['Onboard_date']))
data3n=data2n.withColumn("Month",month(data2n['Onboard_date']))
data4n=data3n.withColumn("dayofmonth",month(data3n['Onboard_date']))

In [50]:
final_results = fit_model.transform(data4n)

In [51]:
final_results.select('Company','Prediction').show()

+----------------+----------+
|         Company|Prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



# Therefore we have predicted which Companies would churn
